# **Dataset Description**

## Dataset: BCI Competition 2008 - Graz dataset A

**Description**: This dataset consists of EEG data from 9 subjects. The cue-based BCI paradigm consisted of four different motor imagery tasks, namely the imagination of movement of the left hand (**class 1**), right hand (**class 2**), both feet (**class 3**), and tongue (**class 4**). Two sessions on different days were recorded for each subject. Each session is comprised of 6 runs separated by short breaks. One run consists of 48 trials (12 for each of the four possible classes), yielding a total of 288 trials per session.

<br />

<img src="https://www.dropbox.com/s/cakz6r7o23iqp4u/Motor_imagery.png?raw=1" alt="Timing scheme of the paradigm" />

<br />

**Experimental paradigm:** After two seconds (t = 2 s), a cue in the form of an arrow pointing either to the left, right, down or up (corresponding to one of the four classes left hand, right hand, foot or tongue) appeared and stayed on the screen for 1.25 s. This prompted the subjects to perform the desired motor imagery task. No feedback was provided. The subjects were ask to carry out the motor imagery task until the fixation cross disappeared from the screen at t = 6 s.

**Input:** *22 EEG signals, 5 seconds long, 250 Hz sampling frequency*

**Input Dimensions:** (288, 22, 1001)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm


def EEGNet(nb_classes, Chans = 64, Samples = 128,
             dropoutRate = 0.5, kernLength = 64, F1 = 8,
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')

    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False,
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)

    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)

    flatten      = Flatten(name = 'flatten')(block2)

    dense        = Dense(nb_classes, name = 'dense',
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)

    return Model(inputs=input1, outputs=softmax)

We define a deep learning model called **EEGNet** for analyzing the EEG data.

The model is tailored to classify EEG signals into different classes. The architecture consists of several layers such as convolutional, pooling, and fully connected layers. EEGNet utilizes techniques like **dropout** and **batch normalization** to enhance the model's performance and prevent overfitting. It takes into account the specific characteristics of EEG data, such as the *number of channels* and *samples per channel*, and can be customized by adjusting parameters like *dropout rate*, *kernel length* and *number of classes*.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import numpy as np


data_file_name = 'bigger.json'
data_file_path = f'/content/drive/My Drive/{data_file_name}'

# Open the JSON file
with open(data_file_path, 'r') as file:
    # Load the JSON data
    loaded_json_data = json.load(file)

# create numpy array 'data_array'
data_array = np.array(loaded_json_data['x'])
# convert array values to float
data_array = np.transpose(data_array, (0, 2, 1)).astype(float)

In [ ]:
data_array.shape
# (trials, channels, features)

(288, 22, 1001)

In [ ]:
num_classes = 4

output_values = loaded_json_data['y']
output_values = np.array([x - 1 for x in output_values])
output_values = np.eye(num_classes)[output_values]
output_values

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

In [ ]:
X_train      = data_array[0:144,]
Y_train      = output_values[0:144]
X_validate   = data_array[144:216,]
Y_validate   = output_values[144:216]
X_test       = data_array[216:,]
Y_test       = output_values[216:]

We split the data to train/validate/test.

In [ ]:
kernels, chans, samples = 1, 22, 1001

X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
X_validate   = X_validate.reshape(X_validate.shape[0], chans, samples, kernels)
X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)

In [ ]:
model = EEGNet(nb_classes = num_classes, Chans = chans, Samples = samples,
               dropoutRate = 0.5, kernLength = 32, F1 = 8, D = 2, F2 = 16,
               dropoutType = 'Dropout')

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics = ['accuracy'])

In [ ]:
numParams = model.count_params()

In [ ]:
fittedModel = model.fit(X_train, Y_train, batch_size = 16, epochs = 300,
                        verbose = 2, validation_data=(X_validate, Y_validate),
                        callbacks=[], class_weight = {})

Epoch 1/300
9/9 - 6s - loss: 1.3890 - accuracy: 0.3403 - val_loss: nan - val_accuracy: 0.2083 - 6s/epoch - 673ms/step
Epoch 2/300
9/9 - 3s - loss: 1.3317 - accuracy: 0.3819 - val_loss: nan - val_accuracy: 0.2778 - 3s/epoch - 320ms/step
Epoch 3/300
9/9 - 2s - loss: 1.2644 - accuracy: 0.4444 - val_loss: nan - val_accuracy: 0.3194 - 2s/epoch - 269ms/step
Epoch 4/300
9/9 - 2s - loss: 1.2288 - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.3611 - 2s/epoch - 267ms/step
Epoch 5/300
9/9 - 2s - loss: 1.1993 - accuracy: 0.4861 - val_loss: nan - val_accuracy: 0.4167 - 2s/epoch - 272ms/step
Epoch 6/300
9/9 - 4s - loss: 1.1715 - accuracy: 0.5278 - val_loss: nan - val_accuracy: 0.4167 - 4s/epoch - 405ms/step
Epoch 7/300
9/9 - 3s - loss: 1.1508 - accuracy: 0.5417 - val_loss: nan - val_accuracy: 0.4306 - 3s/epoch - 378ms/step
Epoch 8/300
9/9 - 2s - loss: 1.1337 - accuracy: 0.5417 - val_loss: nan - val_accuracy: 0.4722 - 2s/epoch - 271ms/step
Epoch 9/300
9/9 - 3s - loss: 1.1179 - accuracy: 0.5764 -